## Sraping data about all US presidental elections (1789-2020)

### Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import seaborn as sns

### Connection

In [2]:
url = "https://www.270towin.com/historical-presidential-elections/"
response = requests.get(url)
response.status_code

200

### Scraping code

In [4]:
PATH = 'C:\webdrivers\chromedriver.exe'
driver = webdriver.Chrome(executable_path=PATH)
driver.get(url)
time.sleep(5)
cookies = driver.find_element(By.XPATH,'//button[@class=" css-47sehv"]')
cookies.click()
driver.implicitly_wait(10)
all_election_links = []
election_years = []
election_data = []
election_links = driver.find_elements(By.XPATH,'.//div[@class="historical-map-thumb"]//a[2]')
for election in election_links:
    election = election.get_attribute('href')
    all_election_links.append({'link':election})
for i in all_election_links:
    driver.get(i["link"])
    driver.implicitly_wait(5)
    driver.refresh()
    time.sleep(3)
    table_data = driver.find_elements(By.XPATH,'.//table[@class="table table-sm table-striped border-bottom"]//tbody//tr')
    election_year = (driver.find_element(By.XPATH,'//h3[@class="mt-4"]').text.strip() + ' ').split(' Election  ') * len(table_data)
    for k in election_year:
        k  = f'{k}'
        election_years.append({'year':k})
    for l in table_data:
        election_candidates = l.find_element(By.XPATH,'.//td[3]').text.strip()
        candidate_party = l.find_element(By.XPATH,'.//td[4]').text.strip()
        electoral_votes = l.find_element(By.XPATH,'.//td[5]').text.strip()
        try:
            popular_votes = l.find_element(By.XPATH,'.//td[6]').text.strip()
        except:
            popular_votes = np.NaN
        election_data.append({'candidates':election_candidates,
                              'party':candidate_party,
                              'electoral_votes':electoral_votes,
                              'popular_votes':popular_votes
                              })
    time.sleep(1)
    driver.refresh()

C:\Users\piotr\AppData\Local\Temp\ipykernel_13876\1165309194.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=PATH)


In [8]:
df1 = pd.DataFrame(election_data)

In [9]:
df2 = pd.DataFrame(election_years)

In [11]:
df3 = pd.merge(df1,df2,left_index=True,right_index=True)

### Final data frame

In [12]:
df3

,candidates,party,electoral_votes,popular_votes,year
0,Joseph R. Biden,Democratic,306,"81,268,867",2020 Election Results
1,Donald J. Trump (I),Republican,232,"74,216,747",2020 Election Results
2,Jo Jorgensen,Libertarian,0,"1,865,720",2020 Election Results
3,Donald J. Trump,Republican,304,"62,984,828",2016 Election Results
4,Hillary R. Clinton,Democratic,227,"65,853,514",2016 Election Results
...,...,...,...,...,...
174,George Clinton,Anti-Federalist,50,NaN,1792 Election Results
175,Others,Anti-Federalist,5,NaN,1792 Election Results
176,George Washington,Federalist,69,NaN,1789 Election Results
177,Others,Independent,35,NaN,1789 Election Results
